## 05-topic-model-merge-docids

**Purpose**: Create topic summary per model and enrich document-level data by merging the `doc_id` associated with each `top_topic` assignment.

In [1]:
import os
import pandas as pd

In [2]:
from usrightmedia.shared.topics_utils import *

### 1.1 Load dataframe containing each `doc_id` and its associated pre-processed text

In [3]:
# df_titles = pd.read_pickle(os.path.join(INPUTS_DIR, "docs", "docs_titles_with_inca_ids.pkl"))
# df_leads = pd.read_pickle(os.path.join(INPUTS_DIR, "docs", "docs_leads_with_inca_ids.pkl"))
df_texts = pd.read_pickle(os.path.join(INPUTS_DIR, "docs", "docs_texts_with_inca_ids.pkl"))

In [4]:
df_texts

,doc_id,processed_doc
0,AmericanRenaissance_1128638341,"[federal, government, study, reparation, desce..."
1,Breitbart_621129461,"[portrait, break, county, headquarters, vandal..."
2,Breitbart_1483020896,"[death, sharply, american, community, multinat..."
3,Breitbart_1483567174,"[ride, giant, courier, service, lawsuit, brake..."
4,AmericanRenaissance_1812166693,"[dark, money, network, life, nearly, taxpayer,..."
...,...,...
727660,WashingtonExaminer_999923116,"[host, border, official, appalling, federal, l..."
727661,WashingtonExaminer_999923435,"[evening, struggle, civil, right, history, tim..."
727662,WashingtonExaminer_999951831,"[cohost, recently, commander, chief, pornograp..."
727663,WashingtonExaminer_999952161,"[federal, judge, person, special, counsel, inv..."


### 1.2 Process dataframes

In [5]:
def process_topics_with_ids(n_topics, df_topics, df_docs_with_ids, export=True):
    """Create topic summary and merge doc_id with topic assignments.
    
    Args:
        n_topics (int)
        df_topics (df)
        df_docs_with_ids (df)
        export (bool)
    
    Returns:
        df_topics_with_ids (df)
    """
    
    print("Show summary of each topic's number, tokens, and document count.")
    df_smry = df_topics.groupby('top_topic').first().reset_index()[['top_topic', 'topic_tokens']]
    df_smry['count'] = df_topics.groupby(['top_topic']).size()
    df_smry['pct_total'] = df_smry['count']/df_smry['count'].sum()*100
    with pd.option_context('display.max_colwidth', None):
        display(df_smry)
        
    # aligning by dataframe row index
    df_topics_with_ids = pd.concat([df_topics, df_docs_with_ids], axis=1)
    df_topics_with_ids['top_topic_pct'] = df_topics_with_ids['top_topic_pct']*100
    
    # check that merge worked correctly
    print("Show dataframe with 'processed_doc': check that 'doc_tokens' from 'df_topics_*' matches 'processed_doc' from 'df_texts'.")
    with pd.option_context('display.max_colwidth', None):
        display(df_topics_with_ids)
    
    print("Show dataframe after dropping 'processed_doc' column.")
    df_topics_with_ids = df_topics_with_ids.drop(['processed_doc'], axis=1)
    
    if export:
        df_smry.to_csv(os.path.join(MODELS_DIR, f"lda_corpus_tfidf_docs_texts_topics_{n_topics}", f"lda_corpus_tfidf_docs_texts_topics_{n_topics}_top_topic_summary.csv"))
        df_smry.to_pickle(os.path.join(MODELS_DIR, f"lda_corpus_tfidf_docs_texts_topics_{n_topics}", f"lda_corpus_tfidf_docs_texts_topics_{n_topics}_top_topic_summary.pkl"))
        df_topics_with_ids.to_pickle(os.path.join(MODELS_DIR, f"lda_corpus_tfidf_docs_texts_topics_{n_topics}", f"lda_corpus_tfidf_docs_texts_topics_{n_topics}_top_topic_with_ids.pkl"))
    
    return df_topics_with_ids

### 1.2.1 `lda_corpus_tfidf_docs_texts_topics_10` 

In [6]:
%%time
df_topics_10 = pd.read_pickle(os.path.join(MODELS_DIR, "lda_corpus_tfidf_docs_texts_topics_10", f"lda_corpus_tfidf_docs_texts_topics_10_top_topic.pkl"))
df_topics_10

CPU times: user 18.2 s, sys: 2.45 s, total: 20.6 s
Wall time: 20.6 s


,top_topic,top_topic_pct,topic_tokens,doc_tokens
0,0.0,0.9111,"police, people, man, white, officer, black, at...","[federal, government, study, reparation, desce..."
1,0.0,0.8842,"police, people, man, white, officer, black, at...","[portrait, break, county, headquarters, vandal..."
2,1.0,0.6980,"coronavirus, virus, health, game, pandemic, va...","[death, sharply, american, community, multinat..."
3,1.0,0.5898,"coronavirus, virus, health, game, pandemic, va...","[ride, giant, courier, service, lawsuit, brake..."
4,2.0,0.6377,"investigation, russian, election, email, campa...","[dark, money, network, life, nearly, taxpayer,..."
...,...,...,...,...
727660,0.0,0.5665,"police, people, man, white, officer, black, at...","[host, border, official, appalling, federal, l..."
727661,0.0,0.9410,"police, people, man, white, officer, black, at...","[evening, struggle, civil, right, history, tim..."
727662,2.0,0.5606,"investigation, russian, election, email, campa...","[cohost, recently, commander, chief, pornograp..."
727663,2.0,0.9006,"investigation, russian, election, email, campa...","[federal, judge, person, special, counsel, inv..."


In [7]:
df_topics_10_with_ids = process_topics_with_ids(10, df_topics_10, df_texts)
with pd.option_context('display.max_colwidth', None):
    display(df_topics_10_with_ids)

Show summary of each topic's number, tokens, and document count.


,top_topic,topic_tokens,count,pct_total
0,0.0,"police, people, man, white, officer, black, attack, military, country, gun",307991,42.325933
1,1.0,"coronavirus, virus, health, game, pandemic, vaccine, state, case, mask, team",143890,19.774209
2,2.0,"investigation, russian, election, email, campaign, information, court, president, report, impeachment",97986,13.465812
3,3.0,"voter, campaign, candidate, election, poll, democratic, president, republican, presidential, vote",108461,14.905348
4,4.0,"market, company, bank, trade, oil, economy, file, price, stock, investor",53026,7.287144
5,5.0,"post, slideshow, post_post, royal, dress, wedding, photo, birthday, native, look",1093,0.150206
6,6.0,"immigration, border, illegal, immigrant, wall, migrant, alien, illegal_immigrant, illegal_alien, bill",12777,1.755890
7,7.0,"cigarette, e_cigarette, tobacco, e, smoking, smoker, vaping, marijuana, nicotine, product",245,0.033669
8,8.0,"storm, hurricane, wind, fire, rain, live, flooding, wildfire, weather, water",2192,0.301238
9,9.0,"cartel, mexican, drug_cartel, drug, gunman, cartel_gunman, drug_trafficking, mexican_cartel, faction, chronicle",4,0.000550


Show dataframe with 'processed_doc': check that 'doc_tokens' from 'df_topics_*' matches 'processed_doc' from 'df_texts'.


,top_topic,top_topic_pct,topic_tokens,doc_tokens,doc_id,processed_doc
0,0.0,91.110001,"police, people, man, white, officer, black, attack, military, country, gun","[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]",AmericanRenaissance_1128638341,"[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]"
1,0.0,88.419998,"police, people, man, white, officer, black, attack, military, country, gun","[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]",Breitbart_621129461,"[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]"
2,1.0,69.800003,"coronavirus, virus, health, game, pandemic, vaccine, state, case, mask, team","[death, sharply, american, community, multinational, automaker, plant, job, foreign, country, late, medical, study, study, acclaimed, researcher, american, community, auto, plant, closure, great, rate, opioid, death, community, auto, plant, open, town, small, city, job, free, trade, opioid, crisis, researcher, county, automotive, assembly, plant, closure, county, automotive, plant, open, automotive, assembly, plant, closure, statistically, significant, 

Show dataframe after dropping 'processed_doc' column.


,top_topic,top_topic_pct,topic_tokens,doc_tokens,doc_id
0,0.0,91.110001,"police, people, man, white, officer, black, attack, military, country, gun","[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]",AmericanRenaissance_1128638341
1,0.0,88.419998,"police, people, man, white, officer, black, attack, military, country, gun","[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]",Breitbart_621129461
2,1.0,69.800003,"coronavirus, virus, health, game, pandemic, vaccine, state, case, mask, team","[death, sharply, american, community, multinational, automaker, plant, job, foreign, country, late, medical, study, study, acclaimed, researcher, american, community, auto, plant, closure, great, rate, opioid, death, community, auto, plant, open, town, small, city, job, free, trade, opioid, crisis, researcher, county, automotive, assembly, plant, closure, county, automotive, plant, open, automotive, assembly, plant, closure, statistically, significant, increase, county, level, opioid, overdose, mortality, rate, adult, emphasis, study, finding, community, auto, plant, opioid, death, community, auto, plant, non, hispanic, white, man, age, hard, opioid, crisis, community, economic, downturn, outsourced, auto, manufacturing, non, hispanic, white, man, great, risk, opioid, death, town, demographic, group, study, study, american, ...]",Breitbart_1483020896
3,1.0,58.980000,"coronavirus, virus, health, game, pandemic, vaccine, state, case, mask, team","[ride, giant, courier, service, lawsuit, brake, new, law, gig, worker, employee, potentially, tech, company, million, dollar, new, personnel, cost, lawsuit, federal, court, law, app, gig, worker, online, company, unfair, treatment, multiple, report, company, law, driver, right, living, manner, choosing, thinly, veiled, attempt, gig, economy, business, irreparably, network, company, app, independent, service, provider, constitutional, right, similarly, complaint, state, tech, giant, new, law, effect, law, potentially, gig, worker, benefit, perk, minimum, wage, sick, pay, attorney, law, company, driver, work, usual, course, business, technology, platform, ridesharing, company, force, counteroffensive, new, law, labor, activist, favor, law, driver, gig, worker, living, wage, benefit, democratic, politician, ...]",Breitbart_1483567174
4,2.0,63.770000,"investigation, russian, election, email, campaign, information, court, president, report, impeachment","[dark, money, network, life, nearly, taxpayer, grant, financial, filing, nonprofit, incubator, government, grant, tax,

- Verify that `docid` content in ES matches the tokens from the dataframe:
```
GET /inca_alias/_search
    {"_source": {
                "excludes": [ "META" ]
            },
            "query": {
                "bool": {
                    "filter": [
                        {"term": {"_id": "AmericanRenaissance_1128638341"}}
                    ]
                }
            }
        }      
```
    
```
GET /inca_alias/_search
    {"_source": {
                "excludes": [ "META" ]
            },
            "query": {
                "bool": {
                    "filter": [
                        {"term": {"_id": "WashingtonExaminer_999952161"}}
                    ]
                }
            }
        }      
```

### 1.2.2 `lda_corpus_tfidf_docs_texts_topics_25` 

In [8]:
df_topics_25 = pd.read_pickle(os.path.join(MODELS_DIR, "lda_corpus_tfidf_docs_texts_topics_25", f"lda_corpus_tfidf_docs_texts_topics_25_top_topic.pkl"))
df_topics_25

,top_topic,top_topic_pct,topic_tokens,doc_tokens
0,0.0,0.9007,"rush, people, white, country, american, right,...","[federal, government, study, reparation, desce..."
1,12.0,0.4578,"police, officer, gun, shooting, protester, man...","[portrait, break, county, headquarters, vandal..."
2,4.0,0.4505,"coronavirus, mask, virus, outbreak, pandemic, ...","[death, sharply, american, community, multinat..."
3,4.0,0.5786,"coronavirus, mask, virus, outbreak, pandemic, ...","[ride, giant, courier, service, lawsuit, brake..."
4,6.0,0.4622,"bill, border, wall, migrant, legislation, fund...","[dark, money, network, life, nearly, taxpayer,..."
...,...,...,...,...
727660,3.0,0.4346,"student, school, sexual, woman, allegation, pr...","[host, border, official, appalling, federal, l..."
727661,0.0,0.6226,"rush, people, white, country, american, right,...","[evening, struggle, civil, right, history, tim..."
727662,3.0,0.8375,"student, school, sexual, woman, allegation, pr...","[cohost, recently, commander, chief, pornograp..."
727663,2.0,0.6899,"investigation, email, russian, campaign, presi...","[federal, judge, person, special, counsel, inv..."


In [9]:
df_topics_25_with_ids = process_topics_with_ids(25, df_topics_25, df_texts)
with pd.option_context('display.max_colwidth', None):
    display(df_topics_25_with_ids)

Show summary of each topic's number, tokens, and document count.


,top_topic,topic_tokens,count,pct_total
0,0.0,"rush, people, white, country, american, right, thing, way, black, world",126264,17.351941
1,1.0,"star, post, fan, film, family, time, photo, movie, good, match",105710,14.527289
2,2.0,"investigation, email, russian, campaign, president, information, impeachment, election, report, intelligence",89871,12.350601
3,3.0,"student, school, sexual, woman, allegation, prison, child, sex, court, assault",42861,5.890210
4,4.0,"coronavirus, mask, virus, outbreak, pandemic, file_photo, case, state, disease, health",51901,7.132540
5,5.0,"medium, white, tweet, black, speech, president, people, racist, video, conservative",69844,9.598373
6,6.0,"bill, border, wall, migrant, legislation, funding, immigration, insurance, budget, plan",31489,4.327403
7,7.0,"military, attack, force, terrorist, troop, government, syrian, war, israeli, minister",37392,5.138628
8,8.0,"voter, poll, candidate, election, campaign, democratic, presidential, vote, republican, primary",46016,6.323789
9,9.0,"vaccine, virus, drug, coronavirus, patient, study, health, disease, death, doctor",12966,1.781864


Show dataframe with 'processed_doc': check that 'doc_tokens' from 'df_topics_*' matches 'processed_doc' from 'df_texts'.


,top_topic,top_topic_pct,topic_tokens,doc_tokens,doc_id,processed_doc
0,0.0,90.070000,"rush, people, white, country, american, right, thing, way, black, world","[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]",AmericanRenaissance_1128638341,"[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]"
1,12.0,45.779999,"police, officer, gun, shooting, protester, man, police_officer, suspect, protest, city","[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]",Breitbart_621129461,"[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]"
2,4.0,45.050003,"coronavirus, mask, virus, outbreak, pandemic, file_photo, case, state, disease, health","[death, sharply, american, community, multinational, automaker, plant, job, foreign, country, late, medical, study, study, acclaimed, researcher, american, community, auto, plant, closure, great, rate, opioid, death, community, auto, plant, open, town, small, city, job, free, trade, opioid, crisis, researcher, county, automotive, assembly, plant, closure, county, automotive, plant, open, automotive, assembly, plant, closure, statisti

Show dataframe after dropping 'processed_doc' column.


,top_topic,top_topic_pct,topic_tokens,doc_tokens,doc_id
0,0.0,90.070000,"rush, people, white, country, american, right, thing, way, black, world","[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]",AmericanRenaissance_1128638341
1,12.0,45.779999,"police, officer, gun, shooting, protester, man, police_officer, suspect, protest, city","[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]",Breitbart_621129461
2,4.0,45.050003,"coronavirus, mask, virus, outbreak, pandemic, file_photo, case, state, disease, health","[death, sharply, american, community, multinational, automaker, plant, job, foreign, country, late, medical, study, study, acclaimed, researcher, american, community, auto, plant, closure, great, rate, opioid, death, community, auto, plant, open, town, small, city, job, free, trade, opioid, crisis, researcher, county, automotive, assembly, plant, closure, county, automotive, plant, open, automotive, assembly, plant, closure, statistically, significant, increase, county, level, opioid, overdose, mortality, rate, adult, emphasis, study, finding, community, auto, plant, opioid, death, community, auto, plant, non, hispanic, white, man, age, hard, opioid, crisis, community, economic, downturn, outsourced, auto, manufacturing, non, hispanic, white, man, great, risk, opioid, death, town, demographic, group, study, study, american, ...]",Breitbart_1483020896
3,4.0,57.860001,"coronavirus, mask, virus, outbreak, pandemic, file_photo, case, state, disease, health","[ride, giant, courier, service, lawsuit, brake, new, law, gig, worker, employee, potentially, tech, company, million, dollar, new, personnel, cost, lawsuit, federal, court, law, app, gig, worker, online, company, unfair, treatment, multiple, report, company, law, driver, right, living, manner, choosing, thinly, veiled, attempt, gig, economy, business, irreparably, network, company, app, independent, service, provider, constitutional, right, similarly, complaint, state, tech, giant, new, law, effect, law, potentially, gig, worker, benefit, perk, minimum, wage, sick, pay, attorney, law, company, driver, work, usual, course, business, technology, platform, ridesharing, company, force, counteroffensive, new, law, labor, activist, favor, law, driver, gig, worker, living, wage, benefit, democratic, politician, ...]",Breitbart_1483567174
4,6.0,46.219997,"bill, border, wall, migrant, legislation, funding, immigration, insurance, budget, plan","[dark, money, network, life, nearly, taxpayer, grant, financial, filing, nonprofit, incubator, governm

### 1.2.3 `lda_corpus_tfidf_docs_texts_topics_40` 

In [10]:
df_topics_40 = pd.read_pickle(os.path.join(MODELS_DIR, "lda_corpus_tfidf_docs_texts_topics_40", f"lda_corpus_tfidf_docs_texts_topics_40_top_topic.pkl"))
df_topics_40

,top_topic,top_topic_pct,topic_tokens,doc_tokens
0,0.0,0.4933,"white, people, rush, black, thing, right, woma...","[federal, government, study, reparation, desce..."
1,0.0,0.5513,"white, people, rush, black, thing, right, woma...","[portrait, break, county, headquarters, vandal..."
2,4.0,0.4378,"market, company, bank, economy, rate, coronavi...","[death, sharply, american, community, multinat..."
3,2.0,0.4516,"company, agency, federal, state, administratio...","[ride, giant, courier, service, lawsuit, brake..."
4,2.0,0.8253,"company, agency, federal, state, administratio...","[dark, money, network, life, nearly, taxpayer,..."
...,...,...,...,...
727660,3.0,0.3685,"sexual, prison, court, woman, police, allegati...","[host, border, official, appalling, federal, l..."
727661,0.0,0.9306,"white, people, rush, black, thing, right, woma...","[evening, struggle, civil, right, history, tim..."
727662,3.0,0.8418,"sexual, prison, court, woman, police, allegati...","[cohost, recently, commander, chief, pornograp..."
727663,6.0,0.4823,"investigation, impeachment, email, russian, pr...","[federal, judge, person, special, counsel, inv..."


In [11]:
df_topics_40_with_ids = process_topics_with_ids(40, df_topics_40, df_texts)
with pd.option_context('display.max_colwidth', None):
    display(df_topics_40_with_ids)

Show summary of each topic's number, tokens, and document count.


,top_topic,topic_tokens,count,pct_total
0,0.0,"white, people, rush, black, thing, right, woman, american, medium, way",151975,20.885297
1,1.0,"game, team, player, fan, season, star, time, home, family, good",128025,17.593948
2,2.0,"company, agency, federal, state, administration, energy, government, technology, system, coal",45414,6.241059
3,3.0,"sexual, prison, court, woman, police, allegation, charge, prosecutor, case, assault",48538,6.670377
4,4.0,"market, company, bank, economy, rate, coronavirus, stock, price, file, file_photo",59091,8.120632
5,5.0,"campaign, candidate, democratic, presidential, election, republican, nominee, voter, primary, president",62932,8.648485
6,6.0,"investigation, impeachment, email, russian, president, campaign, counsel, information, report, election",53789,7.392000
7,7.0,"military, attack, force, terrorist, troop, war, iranian, syrian, government, security",42484,5.838401
8,8.0,"user, company, medium, social_medium, platform, app, social, account, content, ad",20344,2.795792
9,9.0,"bill, tax, legislation, budget, spending, plan, healthcare, insurance, reform, taxis",17153,2.357266


Show dataframe with 'processed_doc': check that 'doc_tokens' from 'df_topics_*' matches 'processed_doc' from 'df_texts'.


,top_topic,top_topic_pct,topic_tokens,doc_tokens,doc_id,processed_doc
0,0.0,49.329998,"white, people, rush, black, thing, right, woman, american, medium, way","[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]",AmericanRenaissance_1128638341,"[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]"
1,0.0,55.129997,"white, people, rush, black, thing, right, woman, american, medium, way","[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]",Breitbart_621129461,"[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]"
2,4.0,43.779999,"market, company, bank, economy, rate, coronavirus, stock, price, file, file_photo","[death, sharply, american, community, multinational, automaker, plant, job, foreign, country, late, medical, study, study, acclaimed, researcher, american, community, auto, plant, closure, great, rate, opioid, death, community, auto, plant, open, town, small, city, job, free, trade, opioid, crisis, researcher, county, automotive, assembly, plant, closure, county, automotive, plant, open, automotive, assembly, plant, closure, statistically, significant, inc

Show dataframe after dropping 'processed_doc' column.


,top_topic,top_topic_pct,topic_tokens,doc_tokens,doc_id
0,0.0,49.329998,"white, people, rush, black, thing, right, woman, american, medium, way","[federal, government, study, reparation, descendant, slave, able, good, way, damage, slavery, african, american, community, lead, sponsor, legislation, fundamental, injustice, cruelty, brutality, inhumanity, slavery, american, colony, commission, national, apology, proposal, reparation, institution, slavery, subsequent, jure, racial, economic, discrimination, impact, force, recommendation, appropriate, remedy, purpose, commission, issue, economic, impact, work, slave, narrative, story, fact, able, damage, recent, interview, speech, urban, blight, school, inner, city, rural, community, level, excellence, support, historically, black, college, university, journey, economic, journey, interesting, magnificent, building, slave, obviously, compensation, bill, commission, people, nation, reparation, legislation, process, educational, process, nation, non, controversial, manner, manner, group, people, snip, strong, supporter, witness, ...]",AmericanRenaissance_1128638341
1,0.0,55.129997,"white, people, rush, black, thing, right, woman, american, medium, way","[portrait, break, county, headquarters, vandal, door, headquarters, portrait, cowboy, hat, mural, flag, elephant, picture, chairman, person, responsible, word, mural, misspelled, slur, gay, people, opps, slang, term, police, motive, mural, office, executive, director, act, vandalism, statement, act, speech, act, destruction, violence, good, possibly, event, local, office, state, contact, local, law, enforcement, protection, volunteer, property, office, headquarters, word, word, office, sign, headquarters, major, damage, inside, building, cowboy_hat, slang_term, executive_director, act_vandalism, law_enforcement]",Breitbart_621129461
2,4.0,43.779999,"market, company, bank, economy, rate, coronavirus, stock, price, file, file_photo","[death, sharply, american, community, multinational, automaker, plant, job, foreign, country, late, medical, study, study, acclaimed, researcher, american, community, auto, plant, closure, great, rate, opioid, death, community, auto, plant, open, town, small, city, job, free, trade, opioid, crisis, researcher, county, automotive, assembly, plant, closure, county, automotive, plant, open, automotive, assembly, plant, closure, statistically, significant, increase, county, level, opioid, overdose, mortality, rate, adult, emphasis, study, finding, community, auto, plant, opioid, death, community, auto, plant, non, hispanic, white, man, age, hard, opioid, crisis, community, economic, downturn, outsourced, auto, manufacturing, non, hispanic, white, man, great, risk, opioid, death, town, demographic, group, study, study, american, ...]",Breitbart_1483020896
3,2.0,45.160000,"company, agency, federal, state, administration, energy, government, technology, system, coal","[ride, giant, courier, service, lawsuit, brake, new, law, gig, worker, employee, potentially, tech, company, million, dollar, new, personnel, cost, lawsuit, federal, court, law, app, gig, worker, online, company, unfair, treatment, multiple, report, company, law, driver, right, living, manner, choosing, thinly, veiled, attempt, gig, economy, business, irreparably, network, company, app, independent, service, provider, constitutional, right, similarly, complaint, state, tech, giant, new, law, effect, law, potentially, gig, worker, benefit, perk, minimum, wage, sick, pay, attorney, law, company, driver, work, usual, course, business, technology, platform, ridesharing, company, force, counteroffensive, new, law, labor, activist, favor, law, driver, gig, worker, living, wage, benefit, democratic, politician, ...]",Breitbart_1483567174
4,2.0,82.529999,"company, agency, federal, state, administration, energy, government, technology, system, coal","[dark, money, network, life, nearly, taxpayer, grant, financial, filing, nonprofit, incubator, government, grant